In [79]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from body_inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq 
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 46.1 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()


In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [8]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [9]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd

In [25]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_tf(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  count = {}
  
  doc_length = 0
  for token in tokens:
    if token not in all_stopwords:
      doc_length += 1 # counting only the words that are not stopwords
      count[token] = count.get(token, 0) + 1 # counting

  if doc_length != 0:
    count_list = []
    for key in count:
      count_list.append((key, (id, (count[key]/doc_length)))) # creating the list and calculating the tf

  return count_list

In [26]:
word_counts = doc_text_pairs.flatMap(lambda x: word_tf(x[0], x[1]))

In [27]:
print(word_counts.take(10))

[('anarchism', (12, 0.029911462072266092)), ('political', (12, 0.006221584111031347)), ('philosophy', (12, 0.0014357501794687725)), ('movement', (12, 0.005503709021296961)), ('sceptical', (12, 0.00023929169657812874)), ('authority', (12, 0.004307250538406317)), ('rejects', (12, 0.00023929169657812874)), ('involuntary', (12, 0.00023929169657812874)), ('coercive', (12, 0.0004785833931562575)), ('forms', (12, 0.0021536252692031586))]


In [28]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl)

In [29]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

In [30]:
print(postings.take(10))

[('anarchism', [(12, 0.029911462072266092), (339, 0.0014038371548900327), (983, 0.00031308703819661864), (1023, 0.01635563011113441)]), ('political', [(12, 0.006221584111031347), (303, 0.0006037918125830214), (307, 0.002949489984023596), (308, 0.0016189088554314393), (336, 0.000257201646090535), (339, 0.004679457182966776), (358, 0.002224199288256228), (359, 0.0007654037504783774), (569, 0.004679457182966776), (594, 9.845426799251748e-05), (600, 0.001530221882172915), (620, 0.002877321475281193), (624, 0.0006886103842445945), (627, 0.0006185567010309278), (643, 0.003257328990228013), (663, 0.0004409171075837742), (665, 0.0012961762799740765), (676, 0.0002961208172934557), (677, 0.000462962962962963), (689, 0.0008726003490401396), (690, 0.0006754474839581223), (691, 0.002352941176470588), (700, 0.0008038585209003215), (701, 0.003659347970725216), (705, 0.004273504273504274), (706, 0.0005434782608695652), (717, 0.0006263701847792045), (736, 0.0004107338444687842), (737, 0.001087744742567

In [31]:
postings_filtered = postings.filter(lambda x: len(x[1])>10)

In [42]:
print(postings_filtered.take(10))

[('political', [(12, 0.006221584111031347), (303, 0.0006037918125830214), (307, 0.002949489984023596), (308, 0.0016189088554314393), (336, 0.000257201646090535), (339, 0.004679457182966776), (358, 0.002224199288256228), (359, 0.0007654037504783774), (569, 0.004679457182966776), (594, 9.845426799251748e-05), (600, 0.001530221882172915), (620, 0.002877321475281193), (624, 0.0006886103842445945), (627, 0.0006185567010309278), (643, 0.003257328990228013), (663, 0.0004409171075837742), (665, 0.0012961762799740765), (676, 0.0002961208172934557), (677, 0.000462962962962963), (689, 0.0008726003490401396), (690, 0.0006754474839581223), (691, 0.002352941176470588), (700, 0.0008038585209003215), (701, 0.003659347970725216), (705, 0.004273504273504274), (706, 0.0005434782608695652), (717, 0.0006263701847792045), (736, 0.0004107338444687842), (737, 0.0010877447425670776), (738, 0.0009751869108245747), (740, 0.0005717552887364208), (746, 0.0012973533990659055), (751, 0.00040306328093510683), (752, 0

In [32]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  def to_df(posting):
    return (posting[0], len(posting[1]))

  return postings.map(to_df)

In [73]:
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

In [34]:
print(w2df.take(10))

[('political', 290), ('philosophy', 135), ('movement', 231), ('sceptical', 12), ('authority', 172), ('rejects', 16), ('involuntary', 12), ('forms', 286), ('hierarchy', 37), ('calls', 105)]


In [63]:
import math
def calculate_idf(postings, doc_count):

  def to_tfidf(posting):
    tfidf_list = map(lambda tup: (tup[0] ,tup[1] * math.log((doc_count/len(posting[1])), 10)), posting[1])
    return ( posting[0], list(tfidf_list)) 
    return (posting[0], math.log(doc_count / posting[1], 10))

  return postings.map(to_tfidf)

In [64]:
postings_filtered_tfidf = calculate_idf(postings_filtered, doc_text_pairs.count())

In [65]:
print(postings_filtered_tfidf.take(10))

[('political', [(12, 0.003344736074330495), (303, 0.0003245996872968505), (307, 0.001585651720588061), (308, 0.000870328641899051), (336, 0.00013827211988195573), (339, 0.0025156855503090493), (358, 0.0011957339904382649), (359, 0.00041148258867282345), (569, 0.0025156855503090493), (594, 5.292921158817011e-05), (600, 0.0008226503475149867), (620, 0.0015468537857994983), (624, 0.00037019832123746303), (627, 0.0003325373208872436), (643, 0.0017511465866483513), (663, 0.00023703791979763834), (665, 0.0006968269631899466), (676, 0.0001591951442407592), (677, 0.0002488898157875203), (689, 0.00046911169467804873), (690, 0.00036312191968999923), (691, 0.0012649458872965736), (700, 0.0004321559502419966), (701, 0.0019672727954462683), (705, 0.0022974444534232643), (706, 0.00029217500114187163), (717, 0.00033673786539370113), (736, 0.00022081133711707713), (737, 0.0005847737513789454), (738, 0.0005242624356820234), (740, 0.00030737678793656023), (746, 0.0006974597847704253), (751, 0.0002166876

In [80]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  '''
  taking a posting of the form (w, posting_list)
  and returning (bucket_id, (w, posting_list))
  '''
  def mapping_bucket_id(posting):
    return (token2bucket_id(posting[0]), posting)
  
  def write_id(id_mapped_postings):
    return InvertedIndex.write_a_posting_list(id_mapped_postings)


  locations = postings.map(mapping_bucket_id).groupByKey().map(write_id)
  
  return locations

In [81]:
posting_locs_list = partition_postings_and_write(postings_filtered_tfidf).collect()

In [82]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [83]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')

In [86]:
TUPLE_SIZE = 8       
TF_MASK = 2 ** 32 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tfidf = unpack('f', b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE])
      posting_list.append((doc_id, tfidf[0]))
    return posting_list

In [87]:
pl = read_posting_list(inverted, 'political')
print(pl)

[(12, 0.003344736061990261), (303, 0.0003245996776968241), (307, 0.0015856517711654305), (308, 0.0008703286293894053), (336, 0.00013827212387695909), (339, 0.002515685511752963), (358, 0.0011957340175285935), (359, 0.0004114825860597193), (569, 0.002515685511752963), (594, 5.2929211960872635e-05), (600, 0.0008226503268815577), (620, 0.0015468538040295243), (624, 0.0003701983077917248), (627, 0.00033253731089644134), (643, 0.0017511466285213828), (663, 0.0002370379224885255), (665, 0.0006968269590288401), (676, 0.00015919514407869428), (677, 0.00024888981715776026), (689, 0.0004691116919275373), (690, 0.0003631219151429832), (691, 0.001264945836737752), (700, 0.00043215593905188143), (701, 0.0019672727212309837), (705, 0.002297444501891732), (706, 0.0002921750128734857), (717, 0.0003367378667462617), (736, 0.00022081134375184774), (737, 0.0005847737775184214), (738, 0.0005242624320089817), (740, 0.00030737678753212094), (746, 0.0006974597927182913), (751, 0.00021668762201443315), (752, 